In [1]:
!pip install -q -r requirements.txt

  DEPRECATION: MarkupSafe is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [64]:
import pandas as pd
import requests
import json

df_voted = pd.read_csv("aanestykset.csv")
df_distr = pd.read_csv("aanestykset_distr.csv")

urls = []

for i, row in df_voted.iterrows():
    r = requests.get(
        f'https://avoindata.eduskunta.fi/api/v1/tables/SaliDBAanestysAsiakirja/rows?perPage=50'
        f'&page=0'
        f'&columnName=AanestysId&columnValue={int(row["AanestysId"])}'
        )
    response_data = json.loads(r.content)

    columns = response_data["columnNames"]
    raw_data = response_data["rowData"]
    
    urls_row = []

    for row in raw_data:
        urls_row.append(row[3])
    
    urls.append(urls_row)

df_voted["info_url"] = urls

df_voted = df_voted[~df_voted.AanestysId.isin([36327.0, 37647.0, 38325.0, 39253.0, 41529.0, 48971.0, 48983.0, 49127.0])]


my_vote = {
    36213: 1,
    36273: 0,
    37601: 1,
    37645: 0,
    41151: 1,
    47469: 1,
    49139: 1,
    49391: 0,
    49445: 1
}

party_vote = {
}


for vote_id in my_vote.keys():
    for i, row in df_distr[df_distr.AanestysId == vote_id].iterrows():
        if not party_vote.get(vote_id):
            party_vote[vote_id] = {}
        votes = int(row["Jaa"])+int(row["Ei"])+int(row["Tyhjia"])+int(row["Poissa"])
        vote_mean = int(row["Jaa"])+int(row["Tyhjia"])*0.5+int(row["Poissa"])*0.5
        party_vote[vote_id][row["Ryhma"]] = vote_mean/votes


parties_res = {}

for vote_id in party_vote:
    for party, vote_val in party_vote[vote_id].items():
        if not parties_res.get(party):
            parties_res[party] = abs(vote_val - my_vote[vote_id])
        else:
            parties_res[party] += abs(vote_val - my_vote[vote_id])

        # print(party +" puolue ja äänestys "+ str(vote_id) + " oli tulos " + str(abs(vote_val - my_vote[vote_id])))

parties_res["Liike Nyt -eduskuntaryhmä"] = parties_res["Liike Nyt -eduskuntaryhmä"]/(9/17)
parties_res["Eduskuntaryhmä Valta kuuluu kansalle"] = parties_res["Eduskuntaryhmä Valta kuuluu kansalle"]/(7/17)
print(parties_res)




# for i, row in df_voted.iterrows():

#     print(row["AanestysId"])
#     for url in row["info_url"]:
#         print("https://www.eduskunta.fi"+url)



{'Keskustan eduskuntaryhmä': 12.292857142857143, 'Kansallisen kokoomuksen eduskuntaryhmä': 12.449146514935991, 'Perussuomalaisten eduskuntaryhmä': 14.131215193521696, 'Sosialidemokraattinen eduskuntaryhmä': 12.425945378151258, 'Vihreä eduskuntaryhmä': 13.091666666666665, 'Vasemmistoliiton eduskuntaryhmä': 12.75, 'Ruotsalainen eduskuntaryhmä': 13.200000000000001, 'Kristillisdemokraattinen eduskuntaryhmä': 13.0, 'Sininen eduskuntaryhmä': 1.0833333333333335, 'Liike Nyt -eduskuntaryhmä': 13.222222222222221, 'Eduskuntaryhmä Valta kuuluu kansalle': 15.785714285714286, 'Eduskuntaryhmä Wille Rydman': 3.0}
